In [3]:
# !pip install datarobot

## Taking an LLM "into" DataRobot

### prequisites

This example relies on Gpt4o, so the requirement is that you have access to Gpt4o.  This example shows 

* the use of Azure OpenAI endpoint that is serving Gpt4o for the purposes of inference.  
* optional tracing with Langsmith. the default behavior is to trace the prompt with langsmith, but you can pass an additional argument called `trace_prompt` as False in order to skip tracing (probably rtecommendation for production)

In order to utilize required credentials,[DataRobot credential manager](https://docs.datarobot.com/en/docs/data/connect-data/stored-creds.html#credentials-management) is being used to keep the Api token.  


## Create `./requirements.txt`

In [1]:
%%writefile ./requirements.txt
langchain == 0.3.13
langchain-community == 0.3.12
langchain-core == 0.3.26
langchain-openai == 0.2.12
langchain-text-splitters == 0.3.4
langsmith == 0.1.147

Writing ./requirements.txt


## Create `./model-metadata.yaml`

in model meda data we provide particulars of the model
* type of model
* runtime parameters, 
* etc.

See the [docs](https://github.com/datarobot/datarobot-user-models/blob/master/MODEL-METADATA.md)

Below, we define the runtime parameters for azure openai and langsmith. 

In [2]:
%%writefile ./model-metadata.yaml 
name: Langsmith Traced OpenAI gpt-4o
type: inference
targetType: textgeneration
runtimeParameterDefinitions:
  - fieldName: AZURE_OPENAI_API_KEY
    type: credential
    credentialType: api_token
    description: Azure OpenAI Api Key
  - fieldName: AZURE_OPENAI_ENDPOINT
    type: string
    defaultValue: https://datarobot-genai-enablement.openai.azure.com/
  - fieldName: PROMPT_COLUMN_NAME
    type: string
    defaultValue: promptText
  - fieldName: LANGCHAIN_API_KEY
    type: credential
    description: token for langsmith tracing
  - fieldName: LANGCHAIN_ENDPOINT
    type: string
    defaultValue: https://api.smith.langchain.com

Writing ./model-metadata.yaml


## create .env file for environment variables for local tests

In [3]:
%%writefile ./.env.template
LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"
LANGCHAIN_API_KEY="<your-api-key>"
AZURE_OPENAI_API_KEY="<api-key>"
AZURE_OPENAI_ENDPOINT="<endpoint>"
OPENAI_API_VERSION=2023-05-15

Writing ./.env.template


In [4]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

## Create `./custom.py`


The following code is what datarobot uses to call your LLM.  

In the code that follows, prompts are passed to the score function as a dataframe.  if `LANGCHAIN_TRACING_V2` is set to False, you can pass through a project name to override the environment setting and complete loggins.  You'll see an example of this later

In [5]:
%%writefile ./custom.py 
from datarobot_drum import RuntimeParameters
from langchain_openai import AzureChatOpenAI
from langchain.callbacks.tracers import LangChainTracer
import pandas as pd
import os

def load_model(*args, **kwargs):
    if (env_key := os.getenv("AZURE_OPENAI_API_KEY")):
        api_key = env_key
    else:
        api_key = RuntimeParameters.get("AZURE_OPENAI_API_KEY")["apiToken"]
    if (env_key:= os.getenv("AZURE_OPENAI_ENDPOINT")):
        azure_endpoint = env_key
    else:
        azure_endpoint = RuntimeParameters.get("AZURE_OPENAI_ENDPOINT")
    if (env_key := os.getenv("LANGCHAIN_API_KEY")):
        # if this passes, it is expected your endpoint is set up accordingly
        pass
    else:
        # pull langchain credentials from runtime parameters
        os.environ["LANGCHAIN_API_KEY"] = RuntimeParameters.get("LANGCHAIN_API_KEY")["apiToken"]
        os.environ["LANGCHAIN_ENDPOINT"] = RuntimeParameters.get("LANGCHAIN_ENDPOINT")

# open
    model = AzureChatOpenAI(
        model="gpt-4o",  # i.e., gpt-4o
        api_key=api_key,
        azure_endpoint=azure_endpoint,
        temperature=0.4,
        api_version="2023-05-15",
    )
    return model

def score(data, model, **kwargs):
    ## langchain tracing is done by passing in additional arguments
    responses = []
    for idx, prompt in data.iterrows():
        if prompt.get("trace_prompt", True):
            if langchain_project := prompt.get("LANGCHAIN_PROJECT", "default"):
                tracer = LangChainTracer(project_name=langchain_project)
                response = model.invoke(prompt.promptText, config={"callbacks": [tracer]})
        else:
            # if LANGCHAIN_TRACING_V2 then logging happens to default project
            response = model.invoke(prompt.promptText)
        responses.append(response.content)
    return pd.DataFrame(dict(resultText=responses))


Overwriting ./custom.py


below, the second prompt is logged to langchain project `math help v3`

In [9]:
import pandas as pd 
from custom import *
data = pd.DataFrame([
    dict(promptText = "tell me a joke", trace_prompt = False),
    dict(promptText = "what's up doc?  can we rock"),
    dict(promptText = "describe bolzano weierstrauss theorem", LANGCHAIN_PROJECT="math help v4"),
    ])

In [10]:
model = load_model(code_dir = ".")
response = score(data, model) 
response

,resultText
0,"Sure, here's a joke for you:\n\nWhy don't skel..."
1,It sounds like you're referencing the classic ...
2,The Bolzano-Weierstrass theorem is a fundament...


## Take this into datarobot

Pick the appropriate datarobot custom model environment

In [15]:
import datarobot as dr
environment = [env for env in dr.ExecutionEnvironment.list() if env.name == "[DataRobot] Python 3.11 GenAI"].pop()
environment


ExecutionEnvironment('[DataRobot] Python 3.11 GenAI')

Create the list of required runtime parameters (see `model-metadata.yaml` for details)
What follows is a list of credentials already being managed by DataRobot.  

In [16]:
credentials = [c for c in dr.Credential.list() if c.name in  ["DR_OPENAI_API_KEY", "Langchain API Key"]]
credentials

[Credential('662811a051f136d67f7997bd', 'DR_OPENAI_API_KEY', 'api_token'),
 Credential('676332535bccdbe998577038', 'Langchain API Key', 'api_token')]

We'll grab the credential ids from the credential list above and set up the run time parameters

In [18]:
runtime_parameter_values = [
    dr.models.runtime_parameters.RuntimeParameterValue(field_name = "AZURE_OPENAI_API_KEY", type = "credential", value = "662811a051f136d67f7997bd"),
    dr.models.runtime_parameters.RuntimeParameterValue(field_name = "LANGCHAIN_API_KEY", type = "credential", value = "676332535bccdbe998577038"),
    ]

Create the custom model in the custom model workshop

In [19]:
cim = dr.CustomInferenceModel.create(name = "Langchain Traced Gpt4o", 
                                    target_type = dr.enums.TARGET_TYPE.TEXT_GENERATION, 
                                    target_name = "resultText", 
                                    network_egress_policy=dr.enums.NETWORK_EGRESS_POLICY.PUBLIC)


add a version with all necessary assets

In [20]:
cimv = dr.CustomModelVersion.create_clean(custom_model_id = cim.id, 
                                          base_environment_id = environment.id, 
                                          runtime_parameter_values = runtime_parameter_values,                                        
                                          files = [("custom.py", "custom.py"),
                                                   ("model-metadata.yaml", "model-metadata.yaml"),
                                                   ("requirements.txt", "requirements.txt")])
## or 
# cim = dr.CustomInferenceModel.get("67350386bab555bb07182a07")
# cimv = dr.CustomModelVersion.create_from_previous(custom_model_id = cim.id, 
#                                                    base_environment_id = environment.id,
#                                                   files = [("custom.py", "custom.py"),
#                                                    ("model-metadata.yaml", "model-metadata.yaml"),
#                                                    ("requirements.txt", "requirements.txt")])
                                        

build the custom model environment (since there was a `requirements.txt` present).  This could take some time

In [21]:
build = dr.CustomModelVersionDependencyBuild.start_build(cim.id, cimv.id, max_wait = 1200)

Register your test dataset

In [22]:
llm_dataset_test = dr.Dataset.create_from_in_memory_data(data_frame = data)
## or 
# llm_dataset_test = dr.Dataset.get("<dataset-id>")

Test the LLM proxy in DataRobot.  This could take some time.  

You should be able to pull up langsmith and check tracing depending on how you set the runtime parameters

In [23]:
custom_model_test = dr.CustomModelTest.create(cim.id, cimv.id, dataset_id = llm_dataset_test.id, network_egress_policy = dr.enums.NETWORK_EGRESS_POLICY.PUBLIC)

In [24]:
custom_model_test.overall_status

'succeeded'

Create a deployment that will be used in the playground

In [25]:
default_prediction_server_id = dr.PredictionEnvironment.list()[-4].id

In [26]:
deployment = dr.Deployment.create_from_custom_model_version(custom_model_version_id = cimv.id, 
                                                            label = "Traced GPT4o", 
                                                            description = "GPT4o traced via langsmith.  Additional arguments in the payload are required to enable tracing", 
                                                            default_prediction_server_id= default_prediction_server_id
                                                            )

## Check that it is working as expected

In [27]:
import requests
import os
import json
import datarobot as dr 
import pprint
# deployment = dr.Deployment.get("673505fb8a477d1f2fbaed3b")
URL = f'{deployment.prediction_environment["name"]}/predApi/v1.0/deployments/{deployment.id}/predictions'
headers = {
    'Content-Type': 'text/plain; charset=UTF-8',
    'Authorization': 'Bearer {}'.format(os.environ["DATAROBOT_API_TOKEN"]),
    'DataRobot-Key': deployment.default_prediction_server["datarobot-key"],
}
query = "what's up doc?"
response = requests.post( URL, headers = headers, data = data.to_csv(index = False))
pprint.pprint(response.json())


{'data': [{'deploymentApprovalStatus': 'APPROVED',
           'prediction': "Sure, here's a joke for you:\n"
                         '\n'
                         'Why don’t skeletons fight each other?\n'
                         '\n'
                         'They don’t have the guts!',
           'predictionValues': [{'label': 'resultText',
                                 'value': "Sure, here's a joke for you:\n"
                                          '\n'
                                          'Why don’t skeletons fight each '
                                          'other?\n'
                                          '\n'
                                          'They don’t have the guts!'}],
           'rowId': 0},
          {'deploymentApprovalStatus': 'APPROVED',
           'prediction': "Sure, we can rock! What's on your mind?",
           'predictionValues': [{'label': 'resultText',
                                 'value': "Sure, we can rock! What's on your "
     

In [28]:
data

,promptText,trace_prompt,LANGCHAIN_PROJECT
0,tell me a joke,False,NaN
1,"what's up doc, can we rock?",NaN,NaN
2,describe bolzano weierstrauss theorem,NaN,math help v4


In [29]:
URL = f'{deployment.prediction_environment["name"]}/predApi/v1.0/deployments/{deployment.id}/predictions'
headers = {
    'Content-Type': 'application/json; charset=UTF-8',
    'Authorization': 'Bearer {}'.format(os.environ["DATAROBOT_API_TOKEN"]),
    'DataRobot-Key': deployment.default_prediction_server["datarobot-key"],
}
query = "what's up doc?"
response = requests.post( URL, headers = headers, 
                         data = json.dumps( [
                             {"promptText": "tell me a joke", "trace_prompt": True, "LANGCHAIN_PROJECT": "comedian"}
                             ]))
pprint.pprint(response.json())


{'data': [{'deploymentApprovalStatus': 'APPROVED',
           'prediction': "Sure, here's a joke for you:\n"
                         '\n'
                         "Why don't skeletons fight each other?\n"
                         '\n'
                         "They don't have the guts!",
           'predictionValues': [{'label': 'resultText',
                                 'value': "Sure, here's a joke for you:\n"
                                          '\n'
                                          "Why don't skeletons fight each "
                                          'other?\n'
                                          '\n'
                                          "They don't have the guts!"}],
           'rowId': 0}]}


## Lasty, 

This last piece will make our LLM available in playground.  Given the way we instrumented tracing in out custom.py file, all prompting done in playground with this llm will be traced and available in the default project for langsmith. 

In [30]:
custom_model_llm_validation = dr.genai.CustomModelLLMValidation.create(
        prompt_column_name="promptText",
        target_column_name="resultText",
        deployment_id=deployment.id,
        wait_for_completion=True, 
        name = deployment.__str__()
    )

assert custom_model_llm_validation.validation_status == "PASSED"